## Project Setup and Imports

In [27]:
# import mysql.connector
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px

import sqlalchemy

## SQLalchemy setup

In [12]:
engine = sqlalchemy.create_engine('mysql+pymysql://root:pw@localhost:3306/formula1')

## Top 20 Drivers by Total Wins

In [59]:
query = '''
SELECT
    CONCAT(dr.forename, ' ', dr.surname) AS name
    , COUNT(re.positionOrder) AS wins
FROM results re
INNER JOIN drivers dr ON dr.driverId = re.driverId
WHERE re.positionOrder = 1
GROUP BY re.driverId
ORDER BY wins DESC
LIMIT 20;
'''

drivers_with_most_wins = pd.read_sql_query(query, engine)

In [141]:
fig = px.bar(drivers_with_most_wins, x='wins', y='name',text='wins', title='F1 Drivers with Most Wins',
    width=1200, height=600)
fig.update_layout(
    yaxis={'categoryorder':'total ascending'},
    margin=dict(l=10, r=20, t=40, b=10))
fig.show()

## Total Wins per Specific Driver

In [63]:
query = '''
SELECT 
    concat(dr.forename, ' ', dr.surname) as name
    , COUNT(re.positionOrder) as wins
FROM results re
INNER JOIN drivers dr ON dr.driverId = re.driverId
WHERE re.positionOrder = 1
'''
wins_by_driver = pd.read_sql_query(query, engine)

In [72]:
driver_name = 'Max Verstappen'

wins_by_driver[wins_by_driver.name == driver_name].wins

105    21
Name: wins, dtype: int64

## Total Wins per Driver for each Constructor

In [87]:
query = '''
SELECT
    c.name AS constructor_name
    , CONCAT(dr.forename, ' ', dr.surname) AS driver_name
    , COUNT(re.positionOrder) AS wins
FROM results re
INNER JOIN drivers dr ON re.driverId = dr.driverId
INNER JOIN constructors c ON re.constructorId = c.constructorId
WHERE re.positionOrder = 1
GROUP BY constructor_name, driver_name
ORDER BY wins desc
;
'''
top_wins_per_driver_constructor = pd.read_sql_query(query, engine)

In [88]:
top_wins_per_driver_constructor

,constructor_name,driver_name,wins
0,Mercedes,Lewis Hamilton,82
1,Ferrari,Michael Schumacher,72
2,Red Bull,Sebastian Vettel,38
3,McLaren,Ayrton Senna,35
4,McLaren,Alain Prost,30
...,...,...,...
183,AlphaTauri,Pierre Gasly,1
184,Racing Point,Sergio PÃ©rez,1
185,Red Bull,Sergio PÃ©rez,1
186,Alpine F1 Team,Esteban Ocon,1


In [154]:
fig = px.bar(top_wins_per_driver_constructor, x='wins', y='driver_name', text='wins', color='constructor_name', 
    title='Wins per Constructor by Driver', width=1200, height=800)

fig.update_layout(
    yaxis={'categoryorder':'total ascending'},
    margin=dict(l=10, r=20, t=40, b=10))

fig.update_traces(width=0.9)

fig.update_yaxes(range=(top_wins_per_driver_constructor.driver_name.nunique()-20.5, top_wins_per_driver_constructor.driver_name.nunique()-0.5))

fig.show()